In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#imports
!pip install pytorch.ignite
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
from ignite.engine import Events, Engine, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import ModelCheckpoint, EarlyStopping
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
#preprocessing transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data_path = "/content/drive/MyDrive/dataset2"
data= {x: datasets.ImageFolder(os.path.join(data_path,x), data_transforms[x]) for x in ['train', 'test']}

In [ ]:
# Create data loaders
data_loaders = {x: DataLoader(data[x], batch_size=60, shuffle=True) for x in ['train', 'test']}
data_size= {x: len(data[x]) for x in ['train', 'test']}
names= data['train'].classes
names

# Define the model
model = models.densenet121(pretrained=True)
in_input = model.classifier.in_features
model.classifier = nn.Linear(in_input, 2)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,  weight_decay=1e-5)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda


In [ ]:
# Create Ignite trainer
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

In [ ]:
metrics = {
    'accuracy': Accuracy(),
    'loss': Loss(criterion)
}
evaluator_train = create_supervised_evaluator(model, metrics=metrics, device=device)
evaluator_test = create_supervised_evaluator(model, metrics=metrics, device=device)

def clear_run_stack(trainer):
    trainer.state.iteration = 0
    trainer.state.epoch = 0
    trainer.state.output = None
    trainer.state.batch = None

trainer.add_event_handler(Events.STARTED, clear_run_stack)

def evaluate_and_check_stopping(engine):
    evaluator_test.run(data_loaders['test'])

early_stopping = EarlyStopping(patience=5, score_function=lambda eng: -eng.state.metrics['loss'], trainer=trainer)
evaluator_test.add_event_handler(Events.COMPLETED, lambda eng: early_stopping(eng))

trainer.add_event_handler(Events.EPOCH_COMPLETED, evaluate_and_check_stopping)

In [ ]:
# Training the model
train_loss_values = []
test_loss_values = []
train_accuracy_values = []
test_accuracy_values = []

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator_train.run(data_loaders['train'])
    metrics = evaluator_train.state.metrics
    accuracy = metrics['accuracy']
    loss = metrics['loss']
    train_accuracy_values.append(accuracy)
    train_loss_values.append(loss)
    print(f"Training Results - Epoch: {trainer.state.epoch}, Accuracy: {accuracy:.2f}, Loss: {loss:.4f}")

@trainer.on(Events.EPOCH_COMPLETED)
def log_test_results(trainer):
    evaluator_test.run(data_loaders['test'])
    metrics = evaluator_test.state.metrics
    accuracy = metrics['accuracy']
    loss = metrics['loss']
    test_accuracy_values.append(accuracy)
    test_loss_values.append(loss)
    print(f"Test Results - Epoch: {trainer.state.epoch}, Accuracy: {accuracy:.2f}, Loss: {loss:.4f}")

# Start the training process
trainer.run(data_loaders['train'], 40)

#DenseNet121 p=10, 40 epochs with early stopping
Training Results - Epoch: 1, Accuracy: 0.87, Loss: 0.3569
Test Results - Epoch: 1, Accuracy: 0.80, Loss: 0.5427
Training Results - Epoch: 2, Accuracy: 0.93, Loss: 0.1812
Test Results - Epoch: 2, Accuracy: 0.93, Loss: 0.1993
Training Results - Epoch: 3, Accuracy: 0.90, Loss: 0.2343
Test Results - Epoch: 3, Accuracy: 0.90, Loss: 0.2622
Training Results - Epoch: 4, Accuracy: 0.91, Loss: 0.2145
Test Results - Epoch: 4, Accuracy: 0.90, Loss: 0.2358
Training Results - Epoch: 5, Accuracy: 0.92, Loss: 0.1850
Test Results - Epoch: 5, Accuracy: 0.91, Loss: 0.2052
Training Results - Epoch: 6, Accuracy: 0.94, Loss: 0.1592
Test Results - Epoch: 6, Accuracy: 0.92, Loss: 0.2241
Training Results - Epoch: 7, Accuracy: 0.93, Loss: 0.1647
Test Results - Epoch: 7, Accuracy: 0.93, Loss: 0.1855
Training Results - Epoch: 8, Accuracy: 0.93, Loss: 0.1666
Test Results - Epoch: 8, Accuracy: 0.93, Loss: 0.1724
Training Results - Epoch: 9, Accuracy: 0.91, Loss: 0.2326
Test Results - Epoch: 9, Accuracy: 0.89, Loss: 0.3070
Training Results - Epoch: 10, Accuracy: 0.94, Loss: 0.1489
Test Results - Epoch: 10, Accuracy: 0.91, Loss: 0.2212
Training Results - Epoch: 11, Accuracy: 0.93, Loss: 0.1699
Test Results - Epoch: 11, Accuracy: 0.90, Loss: 0.2395
Training Results - Epoch: 12, Accuracy: 0.94, Loss: 0.1506
Test Results - Epoch: 12, Accuracy: 0.93, Loss: 0.1823
Training Results - Epoch: 13, Accuracy: 0.94, Loss: 0.1438
Test Results - Epoch: 13, Accuracy: 0.93, Loss: 0.1695
Training Results - Epoch: 14, Accuracy: 0.93, Loss: 0.1714
Test Results - Epoch: 14, Accuracy: 0.93, Loss: 0.1947
Training Results - Epoch: 15, Accuracy: 0.96, Loss: 0.1158
Test Results - Epoch: 15, Accuracy: 0.94, Loss: 0.1489
Training Results - Epoch: 16, Accuracy: 0.95, Loss: 0.1377
Test Results - Epoch: 16, Accuracy: 0.94, Loss: 0.1741
Training Results - Epoch: 17, Accuracy: 0.94, Loss: 0.1507
Test Results - Epoch: 17, Accuracy: 0.91, Loss: 0.2115
Training Results - Epoch: 18, Accuracy: 0.95, Loss: 0.1171
Test Results - Epoch: 18, Accuracy: 0.92, Loss: 0.1985
Training Results - Epoch: 19, Accuracy: 0.94, Loss: 0.1475
Test Results - Epoch: 19, Accuracy: 0.93, Loss: 0.1877
Training Results - Epoch: 20, Accuracy: 0.96, Loss: 0.0983
Test Results - Epoch: 20, Accuracy: 0.94, Loss: 0.1404
Training Results - Epoch: 21, Accuracy: 0.95, Loss: 0.1211
Test Results - Epoch: 21, Accuracy: 0.93, Loss: 0.1757
Training Results - Epoch: 22, Accuracy: 0.95, Loss: 0.1149
Test Results - Epoch: 22, Accuracy: 0.94, Loss: 0.1590
Training Results - Epoch: 23, Accuracy: 0.94, Loss: 0.1498
Test Results - Epoch: 23, Accuracy: 0.91, Loss: 0.2311
Training Results - Epoch: 24, Accuracy: 0.94, Loss: 0.1470
Test Results - Epoch: 24, Accuracy: 0.94, Loss: 0.1716
Training Results - Epoch: 25, Accuracy: 0.96, Loss: 0.1118
2024-04-27 18:13:21,785 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training
Test Results - Epoch: 25, Accuracy: 0.92, Loss: 0.1970
State:
	iteration: 3350
	epoch: 25
	epoch_length: 134
	max_epochs: 40
	output: 0.03878537937998772
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>





In [ ]:
# torch.save(model.state_dict(), 'asmadense_earlystopping.pth')

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = '/content/drive/MyDrive/asmadense_earlystopping.pth'

model = models.densenet121(pretrained=False)
num_features = model.classifier.in_features
model.classifier = torch.nn.Linear(num_features, 2)

model.load_state_dict(torch.load(model_path, map_location=device))

model = model.to(device)


In [ ]:

model.eval()


def plot_confusion_matrix(data_loader, model, device, label_names):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    plt.figure(figsize=(10,10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()
    print('\nPerformance measures:')
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")

train_loss_values = []
test_loss_values = []
train_accuracy_values = []
test_accuracy_values = []

label_names = ['AiArtData', 'RealArt']
plot_confusion_matrix(data_loaders['test'], model, device, label_names)


In [ ]:
# Plotting training and testing process
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss_values) + 1), train_loss_values, label='Training Loss')
plt.plot(range(1, len(test_loss_values) + 1), test_loss_values, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Testing Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_accuracy_values) + 1), train_accuracy_values, label='Training Accuracy')
plt.plot(range(1, len(test_accuracy_values) + 1), test_accuracy_values, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()
plt.show()


